# Bookmark Clustering with Hierarchical Clustering

This notebook demonstrates the process of clustering bookmarks using their embeddings, applying hierarchical clustering, and optimizing the clustering parameters.

In [ ]:
import json
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm.notebook import tqdm

## Helper Functions

In [ ]:
def analyze_embeddings(embeddings):
    print("Embedding analysis:")
    print(f"Shape: {embeddings.shape}")
    print(f"Mean: {np.mean(embeddings)}")
    print(f"Std: {np.std(embeddings)}")
    print(f"Min: {np.min(embeddings)}")
    print(f"Max: {np.max(embeddings)}")
    
    plt.figure(figsize=(10, 5))
    plt.hist(embeddings.flatten(), bins=50)
    plt.title("Distribution of Embedding Values")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.show()

def preprocess_embeddings(embeddings):
    # Center and normalize embeddings
    embeddings_centered = embeddings - np.mean(embeddings, axis=0)
    embeddings_normalized = embeddings_centered / np.linalg.norm(embeddings_centered, axis=1, keepdims=True)
    return embeddings_normalized

def count_bookmarks(folder):
    if folder["type"] == "bookmark":
        return 1
    return sum(count_bookmarks(child) for child in folder.get("children", []))

def print_folder_structure(folder, indent=0):
    if folder["type"] == "bookmark":
        print("  " * indent + f"- {folder['name']} (Bookmark)")
    else:
        bookmark_count = sum(1 for child in folder.get("children", []) if child["type"] == "bookmark")
        subfolder_count = sum(1 for child in folder.get("children", []) if child["type"] == "folder")
        print("  " * indent + f"+ {folder['name']} ({bookmark_count} bookmarks, {subfolder_count} subfolders)")
        for child in folder.get("children", []):
            print_folder_structure(child, indent + 1)

## Clustering Function

In [ ]:
def optimize_clustering(bookmark_data):
    if isinstance(bookmark_data, str):
        bookmark_data = json.loads(bookmark_data)
    
    embeddings = np.array([bookmark["embedding"] for bookmark in bookmark_data])
    
    if embeddings.size == 0:
        raise ValueError("Embeddings are empty. Cannot perform clustering.")
    
    analyze_embeddings(embeddings)
    
    embeddings_preprocessed = preprocess_embeddings(embeddings)
    
    distances = 1 - cosine_similarity(embeddings_preprocessed)
    Z = linkage(distances, method='ward')
    
    plt.figure(figsize=(10, 7))
    dendrogram(Z, truncate_mode='lastp', p=30, leaf_rotation=90., leaf_font_size=12.)
    plt.title('Hierarchical Clustering Dendrogram (truncated)')
    plt.xlabel('sample index or (cluster size)')
    plt.ylabel('distance')
    plt.tight_layout()
    plt.show()
    
    best_score = -1
    best_threshold = None
    best_result = None
    
    for distance_threshold in tqdm(np.arange(0.5, 3.0, 0.1), desc="Optimizing clustering"):
        clusters = fcluster(Z, t=distance_threshold, criterion='distance')
        
        if len(set(clusters)) > 1:
            sil_score = silhouette_score(embeddings_preprocessed, clusters, metric='cosine')
            if sil_score > best_score:
                best_score = sil_score
                best_threshold = distance_threshold
                
                def build_folder_structure(node):
                    if node < len(bookmark_data):
                        return {
                            "name": bookmark_data[node]["title"],
                            "type": "bookmark",
                            "url": bookmark_data[node]["url"]
                        }
                    else:
                        index = node - len(bookmark_data)
                        left = int(Z[index][0])
                        right = int(Z[index][1])
                        folder = {
                            "name": f"Folder {node}",
                            "type": "folder",
                            "children": []
                        }
                        folder["children"].append(build_folder_structure(left))
                        folder["children"].append(build_folder_structure(right))
                        return folder
                
                root_folder = build_folder_structure(len(Z) + len(bookmark_data) - 1)
                
                best_result = {
                    "folder_structure": root_folder,
                    "num_clusters": len(set(clusters)),
                    "distance_threshold": distance_threshold,
                    "silhouette_score": sil_score
                }
    
    return best_result

## Load Data and Perform Clustering

In [ ]:
# Load the embedded bookmarks
with open('embedded_bookmarks.json') as f:
    data = json.load(f)

# Perform clustering
result = optimize_clustering(data)

# Display results
print("Optimized Clustering Results:")
print(f"Number of Clusters: {result['num_clusters']}")
print(f"Optimal Distance Threshold: {result['distance_threshold']:.2f}")
print(f"Best Silhouette Score: {result['silhouette_score']:.4f}")

total_bookmarks = count_bookmarks(result["folder_structure"])
print(f"\nTotal bookmarks: {total_bookmarks}")

print("\nFolder Structure:")
print_folder_structure(result["folder_structure"])

## Visualization of Cluster Sizes

In [ ]:
def get_cluster_sizes(folder):
    if folder["type"] == "bookmark":
        return [1]
    sizes = [count_bookmarks(folder)]
    for child in folder.get("children", []):
        sizes.extend(get_cluster_sizes(child))
    return sizes

cluster_sizes = get_cluster_sizes(result["folder_structure"])

plt.figure(figsize=(12, 6))
plt.hist(cluster_sizes, bins=range(1, max(cluster_sizes) + 2, 1), align='left')
plt.title("Distribution of Cluster Sizes")
plt.xlabel("Cluster Size")
plt.ylabel("Frequency")
plt.xticks(range(1, max(cluster_sizes) + 1, 1))
plt.show()

print(f"Average cluster size: {np.mean(cluster_sizes):.2f}")
print(f"Median cluster size: {np.median(cluster_sizes):.2f}")
print(f"Largest cluster size: {max(cluster_sizes)}")
print(f"Number of singleton clusters: {cluster_sizes.count(1)}")

## Analysis and Conclusions

In this section, you can add your observations about the clustering results, discuss the effectiveness of the hierarchical clustering approach, and suggest potential improvements or next steps.